In [1]:
import pandas as pd
import os
import numpy as np
from Bio import SeqIO
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
input_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/sarscov2/sarscov2_variants_spike_seqences_ncbivirus_20250106.fasta")
output_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/sarscov2/sarscov2_variants_spike_seqences_ncbivirus_20250106.csv")


In [23]:
def parse_fasta_file(input_file_path, output_file_path):
    sequences = []
    i = 0
    parse_error_count = 0
    print("START: Parsing fasta file")
    # parse fasta file to extract uniref90_id, tax_id of virus/organism, and protein sequence
    with open(input_file_path) as f:
        for record in SeqIO.parse(f, "fasta"):
            i += 1
            metadata = record.description.split("|")
            # metadata: 
            # <0> accession id | <1>  protein_with_species | <2> genbank_or_refseq | <3> pangolin | <4> seq_length | <5> protein | <6> host | <7> geo_location | <8> country
            accession_id = metadata[0].strip()
            species = metadata[1].strip()
            genbank_or_refseq = metadata[2].strip()
            pangolin_lineage = metadata[3].strip()
            seq_length = int(metadata[4].strip())
            protein = metadata[5].strip()
            host = metadata[6].strip()
            geo_location = metadata[8].strip()    
        
            
            sequences.append({
                "accession_id": record.id,
                "species": species,
                "genbank_or_refseq": genbank_or_refseq,
                "pangolin_lineage": pangolin_lineage,
                "seq_length": seq_length,
                "protein": protein,
                "host": host,
                "geo_location": geo_location,
                "seq": str(record.seq)
            })
    print("END: Parsing fasta file")
    print(len(sequences))
    print(f"Number of records parsed = {i}")
    print(f"Number of records with parsing error = {parse_error_count}")
    df = pd.DataFrame(sequences)

    # write the parsed dataframe to a csv file
    print(f"Writing to file {output_file_path}")
    df.to_csv(output_file_path, index=False)
    return df

In [ ]:
# df = parse_fasta_file(input_file_path, output_file_path)

In [3]:
df = pd.read_csv(output_file_path)

In [4]:
df

,accession_id,species,genbank_or_refseq,pangolin_lineage,seq_length,protein,host,geo_location,seq
0,YP_009724390.1,surface glycoprotein [Severe acute respiratory...,RefSeq,B,1273,surface glycoprotein,Homo sapiens,China,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,XLZ36708.1,surface glycoprotein [Severe acute respiratory...,GenBank,MC.10.1,1264,surface glycoprotein,Homo sapiens,USA: Iowa,MFVFLVLLPLVSSQCVNLITTTQSYTNFTRGVYYPDKVFRSSVLHL...
2,XLZ36720.1,surface glycoprotein [Severe acute respiratory...,GenBank,XEC,1265,surface glycoprotein,Homo sapiens,USA: Iowa,MFVFLVLLPLVSSQCVNLITTNQSYTNSFTRGVYYPDKVFRSSVLH...
3,XLZ36732.1,surface glycoprotein [Severe acute respiratory...,GenBank,KP.3.1.1,1264,surface glycoprotein,Homo sapiens,USA: Iowa,MFVFLVLLPLVSSQCVNLITTTQSYTNFTRGVYYPDKVFRSSVLHL...
4,XLZ36744.1,surface glycoprotein [Severe acute respiratory...,GenBank,KP.3.1.1,1264,surface glycoprotein,Homo sapiens,USA: Iowa,MFVFLVLLPLVSSQCVNLITTTQSYTNFTRGVYYPDKVFRSSVLHL...
...,...,...,...,...,...,...,...,...,...
4026120,QHN73822.1,"surface glycoprotein, partial [Severe acute re...",GenBank,unclassifiable,35,surface glycoprotein,Homo sapiens,China,NVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDD
4026121,QHN73823.1,"surface glycoprotein, partial [Severe acute re...",GenBank,unclassifiable,35,surface glycoprotein,Homo sapiens,China,NVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDD
4026122,QHN73824.1,"surface glycoprotein, partial [Severe acute re...",GenBank,unclassifiable,35,surface glycoprotein,Homo sapiens,China,NVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDD
4026123,QHO60594.1,surface glycoprotein [Severe acute respiratory...,GenBank,A,1273,surface glycoprotein,Homo sapiens,USA,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [5]:
pd.DataFrame(df["species"].value_counts())

,count
species,
surface glycoprotein [Severe acute respiratory syndrome coronavirus 2],3901811
"surface glycoprotein, partial [Severe acute respiratory syndrome coronavirus 2]",123879
"Surface glycoprotein, partial [Severe acute respiratory syndrome coronavirus 2]",417
spike glycoprotein [Severe acute respiratory syndrome coronavirus 2],7
S protein [Severe acute respiratory syndrome coronavirus 2],5
spike protein [Severe acute respiratory syndrome coronavirus 2],4
"spike protein, partial [Severe acute respiratory syndrome coronavirus 2]",1
RecName: Full=Spike glycoprotein; Short=S glycoprotein; AltName: Full=E2; AltName: Full=Peplomer protein; Contains: RecName: Full=Spike protein S1; Contains: RecName: Full=Spike protein S2; Contains: RecName: Full=Spike protein S2'; Flags: Precursor,1


In [6]:
df = df[df["species"] == "surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]"]

In [7]:
ref_seq_df = df[df["genbank_or_refseq"] == "RefSeq"]
ref_seq_df

,accession_id,species,genbank_or_refseq,pangolin_lineage,seq_length,protein,host,geo_location,seq
0,YP_009724390.1,surface glycoprotein [Severe acute respiratory...,RefSeq,B,1273,surface glycoprotein,Homo sapiens,China,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [8]:
print(df["species"].nunique())
pd.DataFrame(df["species"].value_counts())

1


,count
species,
surface glycoprotein [Severe acute respiratory syndrome coronavirus 2],3901811


In [9]:
print(df["genbank_or_refseq"].nunique())
pd.DataFrame(df["genbank_or_refseq"].value_counts())

2


,count
genbank_or_refseq,
GenBank,3901810
RefSeq,1


In [10]:
print(df["protein"].nunique())
pd.DataFrame(df["protein"].value_counts())

1


,count
protein,
surface glycoprotein,3901811


In [11]:
print(df["host"].nunique())
pd.DataFrame(df["host"].value_counts())

1


,count
host,
Homo sapiens,3901811


In [12]:
print(df["geo_location"].nunique())
pd.DataFrame(df["geo_location"].value_counts())

3669


,count
geo_location,
USA,719372
USA: California,348378
USA: Massachusetts,170200
USA: Florida,156642
USA: Minnesota,139551
...,...
"USA: Massachusetts, Burlington",1
Austria: Upper Austria / Voecklabruck / Niederthalheim,1
Austria: Upper Austria / Voecklabruck / Pilsbach,1


In [13]:
print(df["pangolin_lineage"].nunique())
pangolin_lineage_df = pd.DataFrame(df["pangolin_lineage"].value_counts())
pangolin_lineage_df

3638


,count
pangolin_lineage,
BA.1.1,366872
AY.103,241902
AY.44,207455
B.1.1.7,198709
BA.2.12.1,158794
...,...
ER.1,1
B.1.160.15,1
B.1.212,1


In [14]:
n = df.shape[0]
n_one_percent = 1/100*n
n_one_percent

39018.11

In [15]:
who_designation_df = pd.read_csv(os.path.join(os.getcwd(), "..","..", "..", "..", "input/data/coronaviridae/20240313/sarscov2/sarscov2_who_designated_voc.csv"))
who_designation_df

,pango_lineage,who_variant,designation,first_designation_date,location
0,B,Wuhan-Hu-1,Index,12/01/2019,China
1,B.1.1.7,Alpha,VOC,12/08/2020,United Kingdom
2,B.1.351,Beta,VOC,12/18/2020,South Africa
3,P.1,Gamma,VOC,1/11/2021,Brazil
4,B.1.617.2,Delta,VOC,5/11/2021,India
5,B.1.427,Epsilon,VOI,3/5/2021,USA
6,B.1.429,Epsilon,VOI,3/5/2021,USA
7,P.2,Zeta,VOI,3/17/2021,Brazil
8,B.1.525,Eta,VOI,3/17/2021,Nigeria
9,P.3,Theta,VOI,3/24/2021,Philippines


In [16]:
selected_lineages = list(who_designation_df["pango_lineage"].unique())
selected_lineages

['B',
 'B.1.1.7',
 'B.1.351',
 'P.1',
 'B.1.617.2',
 'B.1.427',
 'B.1.429',
 'P.2',
 'B.1.525',
 'P.3',
 'B.1.526',
 'B.1.617.1',
 'C.37',
 'B.1.621',
 'B.1.1.529',
 'BA.1']

In [44]:
# selected_lineages = list(pangolin_lineage_df[pangolin_lineage_df["count"] > 1000].index)
# print(len(selected_lineages))
# selected_lineages

263


['BA.1.1',
 'AY.103',
 'AY.44',
 'B.1.1.7',
 'BA.2.12.1',
 'AY.3',
 'AY.25',
 'BA.2',
 'BA.1.15',
 'BA.5.2.1',
 'B.1.2',
 'BA.1',
 'AY.100',
 'XBB.1.5',
 'BA.1.1.18',
 'AY.25.1',
 'AY.39',
 'B.1.617.2',
 'BA.5.2',
 'B.1',
 'B.1.526',
 'BA.5.5',
 'BQ.1.1',
 'AY.26',
 'AY.47',
 'BA.5.1',
 'B.1.429',
 'AY.119',
 'AY.122',
 'BA.1.20',
 'BA.4.6',
 'BA.4.1',
 'BA.2.3',
 'BA.1.18',
 'BA.5.6',
 'AY.20',
 'P.1',
 'AY.118',
 'BQ.1',
 'JN.1',
 'BA.2.9',
 'AY.75',
 'HV.1',
 'AY.43',
 'B.1.427',
 'AY.3.1',
 'AY.117',
 'B.1.1.519',
 'KP.3.1.1',
 'BF.10',
 'B.1.637',
 'D.2',
 'AY.14',
 'AY.54',
 'BA.1.17.2',
 'AY.119.2',
 'JN.1.4',
 'unclassifiable',
 'B.1.243',
 'BA.5.2.9',
 'B.1.160',
 'AY.4',
 'BA.2.10',
 'AY.113',
 'FL.1.5.1',
 'BA.5',
 'B.1.1',
 'XBB.1.16.6',
 'BA.5.1.1',
 'AY.39.1',
 'BA.1.1.2',
 'BA.4',
 'BF.7',
 'BE.1',
 'KP.2.3',
 'BF.26',
 'BA.1.17',
 'BF.5',
 'Q.3',
 'XBB.1.16',
 'B.1.351',
 'BE.3',
 'BE.1.1',
 'AY.13',
 'AY.98.1',
 'AY.46.4',
 'AY.99.2',
 'EG.5.1.1',
 'AY.39.1.1',
 'BA.5.

In [17]:
selected_lineages_df = df[df["pangolin_lineage"].isin(selected_lineages)]
selected_lineages_df.shape

(430146, 9)

In [18]:
selected_lineages_df["seq_length"].min()

1253

In [19]:
selected_lineages_df["seq_length"].max()

1281

### Random selection of 100 sequences from each lineage

In [20]:
sampled_dfs = []
for lineage in selected_lineages:
    lineage_df = selected_lineages_df[selected_lineages_df["pangolin_lineage"] == lineage]
    if lineage_df.shape[0] < 100:
        sampled_dfs.append(lineage_df)
    else:
        sampled_dfs.append(lineage_df.sample(n=100, ignore_index=True))
sampled_df = pd.concat(sampled_dfs)
print(sampled_df.shape)
sampled_df

(1525, 9)


,accession_id,species,genbank_or_refseq,pangolin_lineage,seq_length,protein,host,geo_location,seq
0,QUR41397.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,Homo sapiens,Egypt,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,UIX18801.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,Homo sapiens,USA: Illinois,MFVFLVLLPLVSSQCVXXTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,UFJ23379.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,Homo sapiens,USA: Florida,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,UGZ97974.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,Homo sapiens,USA: California,MFVFLVLLPLVSXXCVNLTTRTXXXPAYTNSFTRGVYYPDKVFRSS...
4,UFC65074.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,Homo sapiens,USA: Pennsylvania,MFVFLVLLPLVSSXCVNLTTRTQLPXXXTNSFTXGVYYPDKVFRSS...
...,...,...,...,...,...,...,...,...,...
95,UNN31516.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,Homo sapiens,USA: Florida,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
96,WAA85805.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1272,surface glycoprotein,Homo sapiens,USA,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
97,UIQ58390.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,Homo sapiens,USA: South Carolina,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
98,WDU17786.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,Homo sapiens,Brazil,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [22]:
pd.DataFrame(sampled_df["pangolin_lineage"].value_counts())

,count
pangolin_lineage,
B,100
B.1.1.7,100
B.1.351,100
P.1,100
B.1.617.2,100
B.1.427,100
B.1.429,100
P.2,100
B.1.525,100


In [23]:
variants_df = pd.concat([ref_seq_df, sampled_df])
variants_df = variants_df.rename(columns={"host": "virus_host_name"})
variants_df["virus_host_name"] = variants_df["virus_host_name"].map({"Homo sapiens": "homo sapiens"})
variants_df

,accession_id,species,genbank_or_refseq,pangolin_lineage,seq_length,protein,virus_host_name,geo_location,seq
0,YP_009724390.1,surface glycoprotein [Severe acute respiratory...,RefSeq,B,1273,surface glycoprotein,homo sapiens,China,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
0,QUR41397.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,homo sapiens,Egypt,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,UIX18801.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,homo sapiens,USA: Illinois,MFVFLVLLPLVSSQCVXXTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,UFJ23379.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,homo sapiens,USA: Florida,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,UGZ97974.1,surface glycoprotein [Severe acute respiratory...,GenBank,B,1273,surface glycoprotein,homo sapiens,USA: California,MFVFLVLLPLVSXXCVNLTTRTXXXPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...,...,...,...,...,...,...
95,UNN31516.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,homo sapiens,USA: Florida,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
96,WAA85805.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1272,surface glycoprotein,homo sapiens,USA,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
97,UIQ58390.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,homo sapiens,USA: South Carolina,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
98,WDU17786.1,surface glycoprotein [Severe acute respiratory...,GenBank,BA.1,1270,surface glycoprotein,homo sapiens,Brazil,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [24]:
pd.DataFrame(variants_df["pangolin_lineage"].value_counts())

,count
pangolin_lineage,
B,101
B.1.1.7,100
B.1.351,100
P.1,100
B.1.617.2,100
B.1.427,100
B.1.429,100
P.2,100
B.1.525,100


In [25]:
variants_df.to_csv(os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/sarscov2/sarscov2_variants_spike_seqences_ncbivirus_20250106_who_variants_downsampled.csv"))